In [2]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.lau

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-14 03:57:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.96MB/s    in 0.2s    

2022-10-14 03:57:38 (4.96 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [17]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(50)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...|2015-08-31 00:00:00|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva L

In [22]:
## Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count 
# is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having 
# division by zero errors later on.

totalvotes_df = df.filter("total_votes>=20")
totalvotes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...|2015-08-31 00:00:00|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Ma

In [25]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the 
# rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

helpful50_df = totalvotes_df.filter("helpful_votes / total_votes >= 0.5")
helpful50_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...|2015-08-31 00:00:00|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Ma

In [28]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the 
# rows where a review was written as part of the Vine program (paid), vine == 'Y'.

paid_df = helpful50_df.filter(helpful50_df["vine"] == 'Y')
paid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   34746219| RG70K1HMY4LXX|B010H76PTU|     581545062|Omron 7 Series Wi...|Health & Personal...|          4|           37|         44|   Y|                N|Only tracks 1 per...|This product come...|2015-08-27 00:00:00|
|         US|   51401810|R1DHGTNXDXJ0GB|B00VAPRSP4|     373707856|Celluc

In [30]:
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table and retrieve all the 
# rows where the review was not part of the Vine program (unpaid), vine == 'N'.

unpaid_df = helpful50_df.filter(helpful50_df["vine"] == 'N')
unpaid_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...|2015-08-31 00:00:00|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Ma

In [10]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews
# for the two types of review (paid vs unpaid).

In [36]:
# Total number of paid reviews 
total_reviews_paid_df = paid_df.count()
print("Total Number of Paid Reviews: %f" % total_reviews_paid_df)

Total Number of Paid Reviews: 497.000000


In [40]:
# Total number of unpaid reviews 
total_reviews_unpaid_df = unpaid_df.count()
print("Total Number of Unpaid Reviews: %f" % total_reviews_unpaid_df)

Total Number of Unpaid Reviews: 120863.000000


In [41]:
# Total number of paid 5-star reviews
paid_five_star_reviews = paid_df.filter(paid_df.star_rating == 5).count()
print("Total Number of Paid 5-Star Reviews: %f" % paid_five_star_reviews)

Total Number of Paid 5-Star Reviews: 220.000000


In [44]:
# Total number of unpaid 5-star reviews
unpaid_five_star_reviews = unpaid_df.filter(unpaid_df.star_rating == 5).count()
print("Total Number of Unpaid 5-Star Reviews: %f"  % unpaid_five_star_reviews)

Total Number of Unpaid 5-Star Reviews: 74470.000000


In [45]:
# Percentage of Paid 5-star reviews
paid_percentage_five_star_reviews = paid_five_star_reviews / total_reviews_paid_df * 100
print("Total Percentage of Paid 5-Star Reviews: %f" % paid_percentage_five_star_reviews)

Total Percentage of Paid 5-Star Reviews: 44.265594


In [46]:
# Percentage of Unpaid 5-star reviews
unpaid_percentage_five_star_reviews = unpaid_five_star_reviews / total_reviews_unpaid_df * 100
print("Total Percentage of Unpaid 5-Star Reviews: %f" % unpaid_percentage_five_star_reviews)

Total Percentage of Unpaid 5-Star Reviews: 61.615217
